In [1]:
# import os
# # import Google Drive 套件
# from google.colab import drive
# # 將自己的雲端硬碟掛載上去
# drive.mount('/content/gdrive')

# os.chdir('./gdrive/MyDrive/Colab Notebooks/deep_learning/aoi')      # 檔案目錄

In [2]:
# os.listdir('./../../../../../'), os.getcwd()

In [3]:
# import shutil
# shutil.copyfile('./dataset.zip', './../../../../../dataset.zip')

In [4]:
# import zipfile
# def zip_list(file_path):
#     zf = zipfile.ZipFile(file_path, 'r')
#     zf.extractall('./../../../../../')

# file_path = './../../../../../dataset.zip'
# zip_list(file_path)

In [5]:
# os.remove('./../../../../../dataset.zip')

In [6]:
import torch
import torch.nn as nn
from torch import optim
import numpy as np
import pandas as pd
import os
import pickle
import random
import time
import importlib
import sys
sys.path.append('./GWFM/')

import GWFM.methods.AlgOT as at
import GWFM.methods.FusedGromovWassersteinFactorization as FW
from methods.AlgOT import cost_mat, ot_fgw
from methods.DataIO import StructuralDataSampler, StructuralDataSampler2, structural_data_split
from sklearn.manifold import MDS, TSNE
from typing import List, Tuple
# import functions as fc

In [7]:
!nvidia-smi

Sat Jun 26 19:25:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100S-PCI...  On   | 00000000:0B:00.0 Off |                    0 |
| N/A   49C    P0    41W / 250W |  20959MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Load data

In [8]:
with open('./dataset/datas.pickle', 'rb') as f:
    datas = pickle.load(f)

# define model

In [9]:
modelSave = './models/gwModel_atoms56.pt'
# 訓練超參數 model
data_sampler = StructuralDataSampler2(datas)
num_samples = len(data_sampler)
num_atoms = 56
size_atoms = num_atoms * [35]
ot_method = 'ppa'
gamma = 5e-2
gwb_layers = 5
ot_layers = 30
dim_embedding = 4
num_classes = None       # 先驗分布
prior = None             # 先驗分布

# 訓練超參數 train
size_batch = 16
epochs = 10
lr = 0.1
weight_decay = 0
shuffle_data = True
zeta = 0.1  # the weight of diversity regularizer
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
model = FW.FGWF(num_samples=num_samples,
                num_classes=num_classes,
                size_atoms=size_atoms,
                dim_embedding=dim_embedding,
                ot_method=ot_method,
                gamma=gamma,
                gwb_layers=gwb_layers,
                ot_layers=ot_layers,
                prior=prior)
# model.load_state_dict(torch.load('./models/gwModel1.pt'))

In [11]:
def train_FGWF(model,
              data_sampler,
              size_batch: int = 16,
              epochs: int = 10,
              lr: float = 1e-1,
              weight_decay: float = 0,
              shuffle_data: bool = True,
              zeta: float = None,
              device=None):
    """
    training a FGWF model
    Args:
        model: a FGWF model
        database: a list of data, each element is a list representing [cost, distriubtion, feature, label]
        size_batch: the size of batch, deciding the frequency of backpropagation
        epochs: the number epochs
        lr: learning rate
        weight_decay: the weight of the l2-norm regularization of parameters
        shuffle_data: whether shuffle data in each epoch
        zeta: the weight of the regularizer enhancing the diversity of atoms
        mode: fit or transform
        visualize_prefix: display learning result after each epoch or not
    """
    global modelSave
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)  
#     model.to(device)
    model.train()

#     data_sampler = StructuralDataSampler2(database)
    num_samples = data_sampler.__len__()
    index_samples = list(range(num_samples))
    index_atoms = list(range(model.num_atoms))

    for epoch in range(epochs):
        print('##############{}'.format(epoch))
        counts = 0
        t_start = time.time()
        loss_total = 0
        d_fgw_total = 0
        reg_total = 0
        optimizer.zero_grad()

        if shuffle_data:
            random.shuffle(index_samples)

        for idx in index_samples:
            data = data_sampler.__getitem__(idx)
#             graph = data[0].to(device)
#             prob = data[1].to(device)
#             emb = data[2].to(device)
            graph = data[0]
            prob = data[1]
            emb = data[2]

            # Envelop Theorem
            # feed-forward computation of barycenter B({Ck}, w) and its transports {Trans_k}
            trans = []
            for k in range(model.num_atoms):
                graph_k = model.output_atoms(k).data
                emb_k = model.embeddings[k].data
#                 psk = model.ps[k].to(device)
                psk = model.ps[k]
                _, tran_k = ot_fgw(graph_k, graph, psk, prob,
                                   model.ot_method, model.gamma, model.ot_layers,
                                   emb_k, emb)
                trans.append(tran_k)
            tran = torch.diag(prob[:, 0])

            d_fgw, _, _, _ = model(graph, prob, emb, idx, trans, tran)
            d_fgw_total += d_fgw
            loss_total += d_fgw

            if zeta is not None:
                random.shuffle(index_atoms)
                graph1 = model.output_atoms(index_atoms[0])
                emb1 = model.embeddings[index_atoms[0]]
#                 p1 = model.ps[index_atoms[0]].to(device)
                p1 = model.ps[index_atoms[0]]

                graph2 = model.output_atoms(index_atoms[1])
                emb2 = model.embeddings[index_atoms[1]]
#                 p2 = model.ps[index_atoms[1]].to(device)
                p2 = model.ps[index_atoms[1]]
                _, tran12 = ot_fgw(graph1.data, graph2.data, p1, p2,
                                   model.ot_method, model.gamma, model.ot_layers,
                                   emb1.data, emb2.data)
                reg = FW.fgwd(graph1, emb1, p1, graph2, emb2, p2, tran12)

                reg_total += zeta * reg
                loss_total -= zeta * reg

            counts += 1
            if counts % size_batch == 0 or counts == num_samples:
                if counts % size_batch == 0:
                    num = size_batch
                else:
                    num = counts % size_batch
                loss_total.backward()
                optimizer.step()

                print('-- {}/{} [{:.1f}%], loss={:.4f}, dgw={:.4f}, reg={:.4f}, time={:.2f}s.'.format(
                    counts, num_samples, counts / num_samples * 100.0,
                    loss_total / num, d_fgw_total / num, reg_total / num, time.time() - t_start))

                t_start = time.time()
                loss_total = 0
                d_fgw_total = 0
                reg_total = 0
                optimizer.zero_grad()
        torch.save(model.state_dict(), modelSave)

In [ ]:
train_FGWF(model=model,
           data_sampler=data_sampler,
           size_batch=size_batch,
           epochs=epochs,
           lr=lr,
           weight_decay=weight_decay,
           shuffle_data=shuffle_data,
           zeta=zeta)

##############0
-- 16/2000 [0.8%], loss=0.5265, dgw=0.5448, reg=0.0183, time=17.70s.
-- 32/2000 [1.6%], loss=0.3617, dgw=0.3801, reg=0.0184, time=18.15s.
-- 48/2000 [2.4%], loss=0.2692, dgw=0.2863, reg=0.0171, time=18.05s.
-- 64/2000 [3.2%], loss=0.2268, dgw=0.2439, reg=0.0170, time=18.06s.
-- 80/2000 [4.0%], loss=0.2046, dgw=0.2201, reg=0.0156, time=17.70s.
-- 96/2000 [4.8%], loss=0.1819, dgw=0.1981, reg=0.0162, time=17.31s.
-- 112/2000 [5.6%], loss=0.1847, dgw=0.2015, reg=0.0168, time=17.28s.
-- 128/2000 [6.4%], loss=0.1642, dgw=0.1806, reg=0.0164, time=17.66s.
-- 144/2000 [7.2%], loss=0.1645, dgw=0.1794, reg=0.0149, time=17.24s.
-- 160/2000 [8.0%], loss=0.1480, dgw=0.1660, reg=0.0180, time=18.16s.
-- 176/2000 [8.8%], loss=0.1397, dgw=0.1552, reg=0.0154, time=16.31s.
-- 192/2000 [9.6%], loss=0.1315, dgw=0.1491, reg=0.0175, time=17.33s.
-- 208/2000 [10.4%], loss=0.1330, dgw=0.1496, reg=0.0166, time=17.10s.
-- 224/2000 [11.2%], loss=0.1207, dgw=0.1397, reg=0.0190, time=17.43s.
-- 240/2

-- 1856/2000 [92.8%], loss=0.0235, dgw=0.0704, reg=0.0469, time=17.26s.
-- 1872/2000 [93.6%], loss=0.0250, dgw=0.0735, reg=0.0485, time=17.18s.
-- 1888/2000 [94.4%], loss=0.0286, dgw=0.0789, reg=0.0504, time=17.95s.
-- 1904/2000 [95.2%], loss=0.0395, dgw=0.0977, reg=0.0582, time=17.76s.
-- 1920/2000 [96.0%], loss=0.0287, dgw=0.0787, reg=0.0500, time=18.81s.
-- 1936/2000 [96.8%], loss=0.0120, dgw=0.0673, reg=0.0552, time=16.81s.
-- 1952/2000 [97.6%], loss=0.0277, dgw=0.0801, reg=0.0524, time=17.89s.
-- 1968/2000 [98.4%], loss=0.0301, dgw=0.0776, reg=0.0475, time=18.36s.
-- 1984/2000 [99.2%], loss=0.0190, dgw=0.0742, reg=0.0551, time=16.33s.
-- 2000/2000 [100.0%], loss=0.0093, dgw=0.0652, reg=0.0559, time=16.79s.
##############1
-- 16/2000 [0.8%], loss=0.0290, dgw=0.0819, reg=0.0529, time=17.57s.
-- 32/2000 [1.6%], loss=0.0300, dgw=0.0786, reg=0.0486, time=17.69s.
-- 48/2000 [2.4%], loss=0.0253, dgw=0.0798, reg=0.0544, time=17.72s.
-- 64/2000 [3.2%], loss=0.0301, dgw=0.0877, reg=0.0576, 

In [21]:
model.embeddings[11]

Parameter containing:
tensor([[ 1.2516e+00, -1.4422e-01,  7.3147e+00,  2.0932e-01],
        [ 2.1110e+00, -4.5244e-02,  4.1991e+00,  1.3844e+00],
        [ 1.5844e+00,  1.5836e-01,  6.2744e+00, -1.4201e-01],
        [ 2.5008e+00, -1.1238e-02,  5.7336e+00,  7.2038e-01],
        [ 3.0396e+00,  3.7331e-01,  3.0778e+00, -8.5082e-01],
        [ 2.5251e+00,  1.0760e-01,  4.9875e+00,  1.2279e+00],
        [ 2.8257e+00, -1.3698e-01,  5.2744e+00, -6.7382e-01],
        [ 1.6175e+00,  1.1862e-01,  5.3930e+00,  1.5521e+00],
        [ 3.3539e+00,  2.2564e-01,  6.2408e+00,  1.9255e+00],
        [ 3.0602e+00,  1.3467e-01,  2.8040e+00, -1.2654e+00],
        [ 5.5106e+00, -1.6004e-02,  1.4879e+00,  1.5089e-01],
        [ 2.4532e+00,  2.5648e-02,  4.4223e+00,  1.6875e+00],
        [ 1.7086e+00, -4.1522e-02,  5.2322e+00, -5.5545e-01],
        [ 2.7927e+00, -3.2976e-01,  5.0586e+00,  1.3816e+00],
        [ 2.3055e+00, -3.0619e-01,  4.8395e+00,  1.2780e+00],
        [ 2.5134e+00, -1.8995e-01,  5.0064e+00, 

# try

In [4]:
datas[0]['nodesAttribute']

array([[ 1.        ,  0.        ,  9.77670002, -3.54900002],
       [ 1.        ,  0.        ,  8.91069984, -3.04900002],
       [ 1.        ,  0.        ,  8.91069984, -2.04900002],
       [ 1.        ,  0.        ,  9.77670002, -1.54900002],
       [ 1.        ,  0.        , 10.6427002 , -2.04900002],
       [ 1.        ,  0.        , 10.6427002 , -3.04900002],
       [ 2.        ,  0.        ,  9.77670002, -0.54900002],
       [ 1.        ,  0.        , 10.6427002 , -0.049     ],
       [ 1.        ,  0.        , 11.50879955, -0.54900002],
       [ 1.        ,  0.        , 11.50879955, -1.54900002],
       [ 2.        ,  0.        , 10.6427002 ,  0.95099998],
       [ 2.        ,  0.        ,  8.04469967, -3.54900002],
       [ 1.        ,  0.        ,  7.17859983, -3.04900002],
       [ 1.        ,  0.        ,  7.17859983, -2.04900002],
       [ 1.        ,  0.        ,  8.04469967, -1.54900002],
       [ 1.        ,  0.        ,  7.17859983,  0.95099998],
       [ 1.        ,  0.

In [5]:
cs = torch.tensor(datas[0]['adjecentMatrix'], dtype=torch.float)
ct = torch.tensor(datas[1]['adjecentMatrix'], dtype=torch.float)
embs = torch.tensor(datas[0]['nodesAttribute'], dtype=torch.float)
embt = torch.tensor(datas[1]['nodesAttribute'], dtype=torch.float)
ps = torch.ones(cs.shape[0], 1, dtype=torch.float) / cs.shape[0]
pt = torch.ones(ct.shape[0], 1, dtype=torch.float) / ct.shape[0]
tran = torch.matmul(ps, pt.t())

In [6]:
outputs1 = at.ot_fgw(cs, ct, ps, pt, 'b-admm', 1., 10, embs, embt)     # 計算兩圖的距離
outputs2 = at.ot_fgw(cs, ct, ps, pt, 'b-admm', 1., 10)     # 計算兩圖的距離

In [10]:
outputs1

(tensor(0.2803),
 tensor([[0.0019, 0.0029, 0.0022, 0.0020, 0.0013, 0.0017, 0.0035, 0.0021, 0.0013,
          0.0012, 0.0013],
         [0.0019, 0.0032, 0.0022, 0.0021, 0.0015, 0.0018, 0.0031, 0.0020, 0.0012,
          0.0012, 0.0012],
         [0.0017, 0.0026, 0.0021, 0.0023, 0.0018, 0.0020, 0.0025, 0.0022, 0.0014,
          0.0015, 0.0013],
         [0.0015, 0.0024, 0.0019, 0.0024, 0.0020, 0.0021, 0.0021, 0.0023, 0.0015,
          0.0017, 0.0013],
         [0.0016, 0.0025, 0.0020, 0.0024, 0.0019, 0.0020, 0.0023, 0.0023, 0.0014,
          0.0016, 0.0013],
         [0.0017, 0.0027, 0.0021, 0.0021, 0.0015, 0.0018, 0.0030, 0.0023, 0.0014,
          0.0015, 0.0013],
         [0.0017, 0.0016, 0.0019, 0.0020, 0.0017, 0.0020, 0.0017, 0.0025, 0.0021,
          0.0021, 0.0019],
         [0.0013, 0.0018, 0.0016, 0.0025, 0.0024, 0.0023, 0.0014, 0.0025, 0.0018,
          0.0024, 0.0014],
         [0.0013, 0.0019, 0.0017, 0.0025, 0.0023, 0.0023, 0.0015, 0.0024, 0.0017,
          0.0022, 0.0014],
  